# Recommendation Systems for Movies

You will need access to these class files:

1.   RecommendationSystems-links_small.csv here: https://drive.google.com/file/d/1JReaMKCKaMM8R3pWT4eg7GjcqxVW8VNv/view?usp=share_link
2.   Systems-movies_metadata.csv" here: https://drive.google.com/file/d/1WvSQDiRe0aMTqMhlh_A8nnhtThfxYtNL/view?usp=share_link
3.   Systems-ratings_small.csv" https://drive.google.com/file/d/1SAiSzdPCRePMQz2RrQ7l9cTSG4s6GiBJ/view?usp=share_link

For Reference: Two MovieLens datasets are used here. (link : https://www.kaggle.com/rounakbanik/the-movies-dataset/data)



In [ ]:
#01 Install Dependency
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193672 sha256=2ea163483d820542732078030d3963504c85352bd586c0cbe49d2230c8eef321
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
# 02 importing required libraries

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD #(if this code doesn't run, please remove the "#" before pip install surprise in the previous cell and run the cell)
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

## Content Based Recommendation model

![](https://johnolamendy.files.wordpress.com/2015/10/01.png)

** Limitation of Popularity model ** <br>
It gives the same recommendation to everyone, regardless of the user's personal interest. <br>

_For example:_ If a person who loves romantic movies were to look at Top 15 romantic movies, he wouldn't probably like most of the listed movies. If he were to go one step further and look at movie lists by genre, he wouldn't still be getting the interesting recommendations.

Therefore, let's build a model that computes similarity between movies based on certain metrics and suggests movies that are most similar to a particular movie that a user liked. For that we have to consider metadata (or content), hence, it also known as **Content Based Filtering.**

Two Content Based Recommendation is implemented based on different contents:
1. Description Based (content: Movie Overviews and Taglines)
2. Meta Data Based (content : Movie Cast, Crew, Keywords and Genre)

** Note **: A small movie data set is used due to limit computing power requirements.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 03 Load the small links data file

small_mdf = pd.read_csv('/content/drive/MyDrive/CISD307-Module06-Homework01-JupyterRecommendationSystems-links_small.csv')
small_mdf = small_mdf[small_mdf['tmdbId'].notnull()]['tmdbId'].astype('int')

Before extracting small data set, we need to make sure that the ID column of our main dataframe is clean and of type integer. To do this, let us try to perform an integer conversion of our IDs and if an exception is raised,we will replace the ID with NaN. We will then proceed to drop these rows from our dataframe.

In [ ]:
# 04 Define our convenience function

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
# 05 Load the movies metadata data file

m_df = pd.read_csv('/content/drive/MyDrive/CISD307-Module06-Homework01-JupyterRecommendationSystems-movies_metadata.csv')
m_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
# 06 Data cleanup I

m_df['id'] = m_df['id'].apply(convert_int)

In [ ]:
# 07 Data cleanup II

m_df[m_df['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 08 Data cleanup III

m_df = m_df.drop([19730, 29503, 35587])
m_df['id'] = m_df['id'].astype('int')

In [ ]:
# 09 Data cleanup IV

sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.shape

(9099, 24)

**9099** movies avaiable in our small movies metadata dataset

### 1. Description Based Recommendation

In [ ]:
# 10 Data cleanup V

sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')

#### Compute TF-IDF matrix

TF-IDF: Term Frequency-Inverse Document Frequency

In [ ]:
# 11 Term Frequence-Inverse Document Frequency

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sm_df['description'])
tfidf_matrix.shape

(9099, 268124)

#### Cosine Similarity

The Cosine Similarity is used to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Since the TF-IDF Vectorizer is used, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, sklearn's **linear_kernel** is used instead of cosine_similarities as it's much faster.

In [ ]:
# 12 Cosine similarity

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# 13 Inspect

cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

We now have a pairwise cosine similarity matrix for all the movies in our dataset. The next step is to write a function that returns the 50 most similar movies based on the cosine similarity score.

In [ ]:
# 14 Transform the data representation

sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

In [ ]:
# 15 Declare our get_recommendations function

def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Let us now try and get the top 10 recommendations for a few movies.

In [ ]:
# 16 Get recommendations for The Godfather

get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [ ]:
# 17 Get recommendations for The Dark Knight

get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

We see that for **The Dark Knight**, our system is able to identify it as a Batman film and subsequently recommend other Batman films as its top recommendations. But unfortunately, that is all this system can do at the moment. This is not of much use to most people as it doesn't take into considerations very important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie. Someone who liked **The Dark Knight** probably likes it more because of Nolan and would hate **Batman Forever** and every other substandard movie in the Batman Franchise.

Therefore, we are going to use much more suggestive metadata than **Overview** and **Tagline**. In the next subsection, Let's build a more sophisticated recommender that takes **genre**, **keywords**, **cast** and **crew** into consideration.

## Collaborative Filtering

![](https://cdn-images-1.medium.com/max/1600/1*6_NlX6CJYhtxzRM-t6ywkQ.png)



** Limitation of content based recommendation**:
- It is only capable of suggesting movies which are *close* to a certain movie. That is, it is not capable of capturing interest and providing recommendations across genres.

- It doesn't capture the personal intrest and biases of a user. Anyone querying on model for recommendations based on a movie will receive the same recommendations for that movie, regardless of who he is.

Therefore, in this section, we will use a technique called **Collaborative Filtering** to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

We will use the **Surprise** library that used extremely powerful algorithms like **Singular Value Decomposition (SVD)** to minimise RMSE (Root Mean Square Error) and give great recommendations.

In [ ]:
# 18 Prepare for collaborative recommendations

reader = Reader()


In [ ]:
# 19 Load the ratings small data

ratings = pd.read_csv('/content/drive/MyDrive/CISD307-Module06-Homework01-JupyterRecommendationSystems-ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
# 20 Transform the data

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [ ]:
# 21 Create the SVD model

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.90081285, 0.8996618 , 0.89538244, 0.8935106 , 0.89056471]),
 'test_mae': array([0.69317119, 0.69432809, 0.6906349 , 0.68887463, 0.684309  ]),
 'fit_time': (2.8284823894500732,
  2.1289870738983154,
  2.185920476913452,
  2.9455320835113525,
  2.5195677280426025),
 'test_time': (0.6070766448974609,
  0.2527806758880615,
  0.4437868595123291,
  0.4408082962036133,
  0.39354538917541504)}

We get a mean **Root Mean Sqaure Error** of 0.8965 which is more than good enough right now. Let us now train on dataset and arrive at predictions.

Let us pick user 5000 and check the ratings he has given.

In [ ]:
# 22 Find ratings for user 1

ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [ ]:
# 23 Use the model to make a prediction

svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6654862863292315, details={'was_impossible': False})

For movie with ID 302, we get an estimated prediction of **2.74**. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

## Hybrid Recommender


Below is a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

* **Input:** User ID and the Title of a Movie
* **Output:** Similar movies sorted on the basis of expected ratings by that particular user.

In [ ]:
# 24 Declare our data type conversion function

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
# 25 Load the links-small dataset

id_map = pd.read_csv('/content/drive/MyDrive/CISD307-Module06-Homework01-JupyterRecommendationSystems-links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sm_df[['title', 'id']], on='id').set_index('title')

In [ ]:
# 26 Isolate the indices

indices_map = id_map.set_index('id')

In [ ]:
# 27 Declare our hybrid recommendation function

def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']

    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
# 28 Use the hybrid recommendation function

hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
2059,The Matrix,9079.0,7.9,603,3.439313
975,A Grand Day Out,199.0,7.4,530,3.227215
3360,The Dish,62.0,6.6,5257,2.771507
5044,The Men,18.0,6.5,1882,2.740371
1898,A Simple Plan,191.0,6.9,10223,2.739117
4804,Avalon,93.0,6.8,10881,2.727416
6105,A Trip to the Moon,314.0,7.9,775,2.653854
5229,Ambush,13.0,6.3,49320,2.644208
7460,Green Zone,730.0,6.4,22972,2.598499
2561,Supernova,109.0,4.9,10384,2.591327


In [ ]:
# 29 Use the hybrid recommendation function

hybrid(500, 'Avatar')

,title,vote_count,vote_average,id,est
975,A Grand Day Out,199.0,7.4,530,3.692361
2059,The Matrix,9079.0,7.9,603,3.555866
1898,A Simple Plan,191.0,6.9,10223,3.417309
2910,Pandora and the Flying Dutchman,19.0,6.5,38688,3.300546
4804,Avalon,93.0,6.8,10881,3.288769
3360,The Dish,62.0,6.6,5257,3.127506
538,Hellraiser: Bloodline,111.0,4.9,8766,3.112285
7050,Pride and Glory,243.0,6.3,13150,3.085850
6105,A Trip to the Moon,314.0,7.9,775,3.013300
5044,The Men,18.0,6.5,1882,3.005850


We see that for our hybrid recommender, we get different recommendations for different users although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users.

## Conclusion

In this notebook, I have built 3 different recommendation engines based on different ideas and algorithms. They are as follows:

1. **Content Based Recommender:** I built two content based engines; one that took movie overview and taglines as input and the other which took metadata such as cast, crew, genre and keywords to come up with predictions. We also deviced a simple filter to give greater preference to movies with more votes and higher ratings.
2. **Collaborative Filtering:** I used the powerful Surprise Library to build a collaborative filter based on single value decomposition. The RMSE obtained was less than 1 and the engine gave estimated ratings for a given user and movie.
3. **Hybrid Engine:** I brought together ideas from content and collaborative filterting to build an engine that gave movie suggestions to a particular user based on the estimated ratings that it had internally calculated for that user.